# Ficha de Análise Léxica (Lex)

## Introdução

A análise léxica é o processo de conversão de uma sequência de caracteres numa sequência de *tokens*, em que cada *token* representa uma unidade significativa da linguagem à qual os caracteres pertencem.

Por exemplo:

``` 
F = 32 + 1.8 * C
```

Daria a sequência de tokens:

``` 
'F', '=', '32', '+', '1.8', '*', 'C' 
```

Normalmente associamos aos tokens um nome com significado:

```
'ID', 'ATRIB', 'INT', 'SOMA', 'REAL', 'MUL', 'ID' 
```

Mantendo também o valor do token que irá ser necessário para aqueles que são variáveis:

```
('ID', 'F'), ('ATRIB', '='), ('INT', '32'), ('SOMA', '+'), ('REAL', '1.8'), ('MUL', '*'), ('ID', 'C' 
```

Em Python, podemos fazer análise léxica de várias formas. A que iremos utilizar nas aulas recorre ao módulo Ply, que para além de análise léxica vai-nos permitir fazer análise sintática.

Antes de usar o módulo Ply, precisamos de o instalar. Para isso, podemos usar o comando seguinte:

```sh
$ pip install ply
```

Depois, apenas precisamos de importar a ferramenta `lex.py` no nosso programa:

In [2]:
import ply.lex as lex

A primeira coisa que o nosso analisador léxico (ou *lexer*/*tokenizer*) precisa de ter é uma lista de *tokens*. Como exemplo, vamos definir um *lexer* que lê expressões aritméticas, como "4 * (2 + 3)". Neste exemplo já somos capazes de identificar alguns *tokens*...

In [16]:
#tokens = (
##    'INT',
#    'PLUS',
#    'PO',
##    'PC',
#    'MULT'
#)

A seguir é preciso especificar cada *token*. Por outras palavras, precisamos de definir expressões regulares que permitam ao *tokenizer* identificar os *tokens*. Podemos fazê-lo através de variáveis ou de funções.

In [17]:
#t_PLUS = r'\+'
# duas maneiras diferentes




Se alguma ação de processamento sobre o valor do token for necessária temos de o especiicar numa função.

Por exemplo:

```python
def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)
    return t
```
```python
    #isto é codigo python!!
```

Estas funções recebem um parâmetro que é uma instância de `LexToken`, que por sua vez é um objeto com os atributos:
* **type** - Tipo do token, indicado pelo identificador que associamos ao token;
* **value** - A string que fez match com a expressão regular;
* **lineno** - O número da linha que está a ser processada;
* **lexpos** - Posição relativa do token relativamente ao início do texto que está a ser processado.

Podemos especificar um conjunto de caracteres que o analisador léxico vai ignorar.

In [6]:
t_ignore = ' \t\n'#ignorar os espaços

Precisamos ainda de definir o comportamento do *tokenizer* caso encontre um carácter ou sequência de caracteres que não corresponda a nenhum *token* conhecido.

In [7]:
def t_error(t):
    print(f"Carácter ilegal {t.value[0]}")
    t.lexer.skip(1)

Agora, já somos capazes de construir o nosso analisador léxico.

In [11]:
lexer = lex.lex()

TypeError: <module '__main__'> is a built-in module

Para o usar, precisamos de lhe dar algum valor de *input* e depois pedir-lhe para ir devolvendo os *tokens* que encontrar.

In [12]:
data = '''
3 + 4 * 10
  + -20 *2
'''

lexer.input(data)

while tok := lexer.token():
    print(tok)

NameError: name 'lexer' is not defined

Se quisermos manter informação sobre as linhas nas quais os *tokens* aparecem, podemos usar o atributo `lineno`.

In [13]:
t_ignore = ' \t'

def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

### Debug

Podemos ativar esta opção para obtermos mais documentação sobre a geração do analisador léxico:

```python
lexer = lex.lex(debug=True)
```

### Modularização

É possível isolar o analisador léxico num módulo:

```python
# módulo: o meu analisador léxico - analex.py

tokens = (...)
...
def t_error(t):
    ...
```

E depois usá-lo num programa da forma tradicional, ou na forma preparada no `Ply`:

```python 
import analex

lexer = lex.lex(module=analex)
lexer.input(...)
for tok in lexer:
    print(tok)
```

É possível consultar a documentação do *lex.py* em https://ply.readthedocs.io/en/latest/ply.html#lex.

Juntando tudo...

In [ ]:
# Exemplo das expressões aritméticas

import ply.lex as lex # temos 3 tipos: sinais | palavras reservadas | variaveis

tokens = (
    'INT',
    'REAL',
    'ID',
    'ATRIB',
    'SOMA',
    'MUL'
)

t_SOMA = r'\+'
t_MUL = r'\*'
t_ATRIB = r'='
# completar...

def t_ID(t):
    r'[_a-zA-Z]\w*'
    return t
def t_INT(t):
    r'\d+'
    return t
def t_REAL(t):
    r'\d+\.\d+'
    return t

t_ignore = ' \t\n'

def t_error(t):
    print(f"Carácter ilegal {t.value[0]}")
    t.lexer.skip(1)

lexer = lex.lex()

# -------------------------------------------

data = '''
3 + 4 * 10
  + -20 *2
F = 32 + 1.8*C
'''

lexer.input(data)

for tok in lexer:
    print(tok)

ERROR: Rule 't_PLUS' defined for an unspecified token PLUS
ERROR: Rule 't_READ' defined for an unspecified token READ
ERROR: Rule 't_DIV' defined for an unspecified token DIV
ERROR: Rule 't_PA' defined for an unspecified token PA
ERROR: Rule 't_PF' defined for an unspecified token PF
ERROR: Rule 't_VIRG' defined for an unspecified token VIRG


TypeError: <module '__main__'> is a built-in module

### Prioridades:

Quando o `Ply`junta tudo para construir a expressão regular que irá fazer o reconhecimento de todos os tokens segue as seguintes regras:
1. Os tokens definidos por funções são colocados em primeiro lugar pela ordem da definição das funções;
2. Os tokens definidos por strings são colocados a seguir ordenados por ordem decrescente do tamanho da respetiva expressão regular.

## Exercícios

### Aquecimento 

Especifica um analisador léxico para este excerto de uma linguagem de programação:

```
? a
? b
c = a*b/2
d = f(c,2)
! d
```

In [14]:
texto = """
? a
? b
c = a*b/2
d = f(c,2)
"""

import ply.lex as lex

tokens = (
   'NUM',
   'MUL',
   'DIV',
   'PA',
   'PF',
   'READ',
   'ATRIB',
   'VIRG',
   'ID'
)

# Regular expression rules for simple tokens
t_READ    = r'\?'
t_ATRIB   = r'='
t_MUL     = r'\*'
t_DIV     = r'\/'
t_PA      = r'\('
t_PF      = r'\)'
t_VIRG    = r','

# A regular expression rule with some action code
def t_NUM(t):
    r'\d+'
    t.value = int(t.value)
    return t

def t_ID(t):
    r'[a-zA-Z_]\w*'
    return t

# Define a rule so we can track line numbers
def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

# A string containing ignored characters (spaces and tabs)
t_ignore  = ' \t'

# Error handling rule
def t_error(t):
    print(f"Illegal character {t.value[0]}")
    t.lexer.skip(1)

# Build the lexer
lexer = lex.lex()

lexer.input(texto)
for tok in lexer:
    print(tok)

ERROR: Rule 't_PLUS' defined for an unspecified token PLUS


TypeError: <module '__main__'> is a built-in module

## Desafio: colocar o exemplo anterior a processar o stdin

* Isola o analisador léxico num módulo Python;
* Cria um programa que importa o módulo e processa o stdin.

In [ ]:
# Processando o stdin:

### 1. Frases

Define um analisador léxico capaz de ler uma frase e de identificar os seus componentes (palavras, vírgulas, sinais de pontuação).

### 2. Listas Mistas

Define um analisador léxico capaz de receber listas com números, palavras ou valores booleanos como input (e.g.: `[ 1,5, palavra, False ,3.14,   0, fim]`) e identificar os seus *tokens*.

```python

r'\"[^"]*\"' #para as palavras

```

### 3. JSON

Define um analisador léxico capaz de ler ficheiros em formato JSON e identificar os seus *tokens*.

Exemplo de um documento JSON:

---

```json
{
  "name": "John Doe",
  "age": 21,
  "gender": "male",
  "height": 1.68,
  "address": {
    "street": "123 Main Street",
    "city": "New York",
    "country": "USA",
    "zip": "10001"
  },
  "married": false,
  "hobbies": [
    {
      "name": "reading",
      "books": [
        {
          "title": "Heartstopper: Volume 1",
          "author": "Alice Oseman",
          "genres": ["Graphic Novels", "Romance", "Queer"]
        },
        {
          "title": "1984",
          "author": "George Orwell",
          "genres": ["Science Fiction", "Dystopia", "Politics"]
        }
      ]
    },
    {
      "name": "gaming",
      "games": [
        {
          "title": "Portal 2",
          "platform": ["PC", "PlayStation 3", "Xbox 360"]
        },
        {
          "title": "Synth Riders",
          "platform": ["PSVR", "PSVR2", "PCVR", "Oculus Quest"]
        }
      ]
    }
  ]
}
```
---

## Condições de contexto

Para certos analisadores léxico, pode ser útil ter diferentes estados. Por exemplo, se definirmos um analisador léxico para um ficheiro XML, pode ser útil verificar se o nome usado para fechar uma *tag* foi o mesmo que foi usado para a abrir.

Exemplo de parte de um ficheiro XML:

```xml
<pessoa>
    <nome>Maria</nome>
    <idade>32</idade>
</pessoa>
```

In [ ]:
import ply.lex as lex

states = (
    ('taga', 'exclusive'),
    ('tagf', 'exclusive'), # num estado exclusivo, apenas aplicamos os tokens e regras para esse estado
                           # por outro lado, num estado inclusivo, as regras e tokens desse estado juntam-se às outras regras e tokens
                           # o estado inicial chama-se 'INITIAL' e não é preciso defini-lo
)

tokens = (
    'ABRIR_TAG',
    'ABRIR_TAG_F',
    'FECHAR_TAG',
    'NOME_TAG',
    'VALOR'
)

t_ignore = ' \t\n' # estes tokens apenas são ignorados no estado 'INITIAL' e em estados inclusivos

t_VALOR = r'[^<]+'

def t_ABRIR_TAG_F(t):
    r'</'
    t.lexer.begin('tagf') # entramos no estado 'tagf'
    return t

def t_ABRIR_TAG(t):
    r'<'
    t.lexer.begin('taga') # entramos no estado 'taga'
    return t

def t_taga_tagf_FECHAR_TAG(t):
    r'>'
    t.lexer.begin('INITIAL') # voltamos ao estado inicial
    return t

def t_taga_NOME_TAG(t):
    r'\w+'
    t.lexer.stack.append(t.value)
    return t

def t_tagf_NOME_TAG(t):
    r'\w+'
    if len(t.lexer.stack) > 0:
        if (nt := t.lexer.stack.pop(-1)) != t.value:
            print(f"Erro - esperado nome de tag '{nt}', mas foi lido '{t.value}'!")
    else:
        print("Erro - nenhuma tag aberta!")
    return t

def t_ANY_error(t): # regra válida para todos os estados
    print(f"Carácter ilegal: {t.value[0]}")
    t.lexer.skip(1)


data = '''
<pessoa>
    <nome>Maria</nome>
    <idade>32</idade>
</pessoa>
'''

lexer = lex.lex()

lexer.stack = list() # vamos usar esta lista como stack para verificar os nomes das tags

lexer.input(data)

while tok := lexer.token():
    print(tok)

ERROR: Rule 't_PLUS' defined for an unspecified token PLUS
ERROR: Rule 't_READ' defined for an unspecified token READ
ERROR: Rule 't_MUL' defined for an unspecified token MUL
ERROR: Rule 't_DIV' defined for an unspecified token DIV
ERROR: Rule 't_PA' defined for an unspecified token PA
ERROR: Rule 't_PF' defined for an unspecified token PF
ERROR: Rule 't_SOMA' defined for an unspecified token SOMA
ERROR: Rule 't_ATRIB' defined for an unspecified token ATRIB
ERROR: Rule 't_VIRG' defined for an unspecified token VIRG


TypeError: <module '__main__'> is a built-in module

: 

## Exercícios 2

### 1. BibTeX

Define um analisador léxico capaz de ler um ficheiro no formato *BibTeX* e identificar os seus *tokens*.

Exemplo de um ficheiro BibTeX:

---

```bibtex
@incollection {HDYE78,
author = "Ricardo Martini and Pedro Rangel Henriques and Giovani Libreloto",
title = "Storing Archival Emigration Documents to Create Virtual Exhibition Rooms",
booktitle = "New Contributions in Information Systems and Technologies",
series="Advances in Intelligent Systems and Computing",
editor="Rocha, Alvaro and Correia, Ana and Costanzo, S. and Reis, Luis Paulo",
volume="353",
pages="403-409",
year = "2015",
month =  "April"
}


@book {H787,
author = {Vitor T. Martins and Pedro Rangel Henriques and Daniela da Cruz},
title = {An AST-based tool, Spector, for Plagiarism Detection},
booktitle = {Proceedings of SLATE’15},
pages = {173--178},
ISBN = {},
year = {2015},
month =   {},
publisher = {Fundacion General UCM},
annote = {Keywords: software, plagiarism, detection, comparison, test}}

@book {H787,
author = {Vitor T. Martins and Pedro Rangel Henriques and Daniela da Cruz},
title = "{A}n {AST}-based tool, {S}pector, for Plagiarism Detection",
booktitle = {Proceedings of SLATE’15},
pages = {173--178},
ISBN = {},
year = {2015},
month =   {},
publisher = {Fundaci ́on General UCM},
annote = {Keywords: software, plagiarism, detection, comparison, test}
}
```
---

### 2. Somador on/off

Usando um analisador léxico com condições de contexto, cria um programa em Python que tenha o seguinte comportamento:

* Pretende-se um programa que some todas as sequências de dígitos que encontre num texto;
* Prepara o programa para ler o texto do canal de entrada: stdin;
* Sempre que encontrar a string “Off” em qualquer combinação de maiúsculas e minúsculas, esse comportamento é desligado;
* Sempre que encontrar a string “On” em qualquer combinação de maiúsculas e minúsculas, esse comportamento é novamente ligado;
* Sempre que encontrar o caráter “=”, o resultado da soma é colocado na saída.

### 3. Removedor de Comentários

Desenvolve um analisador léxico capaz de ler um ficheiro de texto e ignorar todo o texto dentro de comentários inline (desde "//" até ao fim de linha) e todo o texto dentro de comentários multiline (desde "/*" até "*/").

O *lexer* deve suportar convenientemente comentários dentro de comentários,   conforme exemplificado abaixo:

---
```c
/* comment */ ola1

/* comment****comment */ ola2 /*
comment
/* comentário dentro de comentário */
****/ ola3

/*********/

ola4
 mais um pouco // remover comentário inline
FIM
```
----
